Rough sketch of a proposal

- probably wrong for conjugate
- only optimized allocations and contract! (tensoradd! and tensortrace! were not modified)
- probably fails for non-symmetric tensormaps

In [1]:
using Revise, TensorKit, TensorOperations, BenchmarkTools, Strided, TupleTools, AllocationKit, MPSKitExperimental

In [2]:
import LinearAlgebra
LinearAlgebra.BLAS.set_num_threads(1)
Strided.set_num_threads(1)

1

In [3]:
using Base.Threads
nthreads()

8

# practical benchmarks

In [4]:
@tightloop_tensor ac_eff y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]
@tightloop_tensor ac_eff_malloc allocator=malloc y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]

(ex.head, ex.args) = (:call, Any[:(TensorOperations.promote_contract), :(TensorOperations.scalartype(le)), :(TensorOperations.scalartype(x))])
(ex.head, ex.args) = (:call, Any[:(TensorOperations.promote_contract), :(TensorOperations.scalartype(var"####y_A#292_A#293")), :(TensorOperations.scalartype(O))])
(ex.head, ex.args) = (:call, Any[:(TensorOperations.tensorfree!), Symbol("####y_A#292_A#293")])
(ex.head, ex.args) = (:call, Any[:(TensorOperations.promote_contract), :(TensorOperations.scalartype(var"##y_A#292")), :(TensorOperations.scalartype(re))])
(ex.head, ex.args) = (:call, Any[:(TensorOperations.tensorfree!), Symbol("##y_A#292")])
(ex.head, ex.args) = (:call, Any[:(TensorOperations.promote_contract), :(TensorOperations.scalartype(le)), :(TensorOperations.scalartype(x))])
(ex.head, ex.args) = (:call, Any[:(TensorOperations.promote_contract), :(TensorOperations.scalartype(var"####y_A#310_A#311")), :(TensorOperations.scalartype(O))])
(ex.head, ex.args) = (:call, Any[:(TensorOperati

In [7]:
@tightloop_planar tac_eff y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]

(ex.head, ex.args) = (:tuple, Any[Symbol("##354"), Symbol("##355")])
(ex.head, ex.args) = (:tuple, Any[:(numout(le)), :(numin(le))])
(ex.head, ex.args) = (:||, Any[:(numout(le) == 2 && (TensorKit.numin)(le) == 1), :(throw(TensorOperations.IndexError("incorrect number of input-output indices: (2, 1) instead of " * string((var"##354", var"##355")) * " for le.")))])
(ex.head, ex.args) = (:tuple, Any[Symbol("##356"), Symbol("##357")])
(ex.head, ex.args) = (:tuple, Any[:(numout(O)), :(numin(O))])
(ex.head, ex.args) = (:||, Any[:(numout(O) == 2 && (TensorKit.numin)(O) == 2), :(throw(TensorOperations.IndexError("incorrect number of input-output indices: (2, 2) instead of " * string((var"##356", var"##357")) * " for O.")))])
(ex.head, ex.args) = (:tuple, Any[Symbol("##358"), Symbol("##359")])
(ex.head, ex.args) = (:tuple, Any[:(numout(x)), :(numin(x))])
(ex.head, ex.args) = (:||, Any[:(numout(x) == 2 && (TensorKit.numin)(x) == 1), :(throw(TensorOperations.IndexError("incorrect number of input-

In [39]:
@planar y[-1 -2;-3] := t_le[-1 2;1]*t_o[2 -2;3 4]*t_ac[1 3;5]*t_re[5 4;-3];

In [33]:
virtspace = Rep[SU₂](i => 20 for i in 0:10);
ospace = Rep[SU₂](0 => 5,1 => 2);
pspace = Rep[SU₂](1 => 1);

t_le = TensorMap(rand,ComplexF64,virtspace*ospace',virtspace);
t_re = TensorMap(rand,ComplexF64,virtspace*ospace,virtspace);
t_ac = TensorMap(rand,ComplexF64,virtspace*pspace,virtspace);
t_o = TensorMap(rand,ComplexF64,ospace*pspace,pspace*ospace);

factory = tac_eff(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(t_ac),space(t_ac)));

In [41]:
norm(factory(le = t_le, re=t_re, x = t_ac, O = t_o)-y)

1.1559567890249869e-9

In [35]:
virtspace = Rep[SU₂](i => 20 for i in 0:10);
ospace = Rep[SU₂](0 => 5,1 => 2);
pspace = Rep[SU₂](1 => 1);

t_le = TensorMap(rand,ComplexF64,virtspace*ospace',virtspace);
t_re = TensorMap(rand,ComplexF64,virtspace*ospace,virtspace);
t_ac = TensorMap(rand,ComplexF64,virtspace*pspace,virtspace);
t_o = TensorMap(rand,ComplexF64,ospace*pspace,pspace*ospace);

factory = ac_eff(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(t_ac),space(t_ac)));
factory_malloc = ac_eff_malloc(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(t_ac),space(t_ac)));

In [36]:
factory(le = t_le, re=t_re, x = t_ac, O = t_o);

In [37]:
@benchmark $factory_malloc(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 2114 samples with 1 evaluation.
 Range (min … max):  1.973 ms …   8.399 ms  ┊ GC (min … max): 0.00% … 61.57%
 Time  (median):     2.245 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.338 ms ± 594.143 μs  ┊ GC (mean ± σ):  2.17% ±  6.42%

  ▁█▅▃▄                                                        
  ██████▆▄▃▂▃▂▂▂▂▂▂▂▂▁▂▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂ ▃
  1.97 ms         Histogram: frequency by time        7.12 ms <

 Memory estimate: 476.48 KiB, allocs estimate: 3429.

In [38]:
@benchmark $factory(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 1358 samples with 1 evaluation.
 Range (min … max):  2.566 ms … 9.936 ms  ┊ GC (min … max):  0.00% … 47.18%
 Time  (median):     2.870 ms             ┊ GC (median):     0.00%
 Time  (mean ± σ):   3.646 ms ± 1.562 ms  ┊ GC (mean ± σ):  17.96% ± 19.90%

  ▅██▇▆▄▂▂▂▁                              ▂▂▂▃▃▁▂           ▁
  ████████████▆▇▇▄▅▆▇▄▁▅▁▁▁▅▁▁▁▁▄▁▁▁▁▁▁▁▅▇██████████▇█▇▅▅▅▅ █
  2.57 ms     Histogram: log(frequency) by time     7.72 ms <

 Memory estimate: 8.56 MiB, allocs estimate: 3422.

In [14]:
@benchmark $factory(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

BenchmarkTools.Trial: 1015 samples with 1 evaluation.
 Range (min … max):  3.356 ms … 9.390 ms  ┊ GC (min … max):  0.00% … 35.75%
 Time  (median):     3.894 ms             ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.891 ms ± 1.691 ms  ┊ GC (mean ± σ):  18.21% ± 18.93%

   ▃█▆▅▃▂                                                    
  ▆██████▆▅▃▃▃▃▄▃▃▃▃▃▃▃▂▂▁▁▂▁▁▁▁▂▁▁▁▁▁▁▁▂▃▂▃▃▅▄▆▆▆▆▄▄▅▃▃▃▂▃ ▃
  3.36 ms        Histogram: frequency by time       8.25 ms <

 Memory estimate: 12.65 MiB, allocs estimate: 5093.

In [ ]:
function slowcontract(;le=nothing,re=nothing,x=nothing,O=nothing)
    @tensor y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]
end
@benchmark slowcontract(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

In [ ]:
norm(slowcontract(le = t_le, re=t_re, x = t_ac, O = t_o)- factory(le = t_le, re=t_re, x = t_ac, O = t_o))

In [ ]:
virtspace = Rep[SU₂×U₁×ℤ₂]((i,j,b) => 10 for i in 0:5,j in -3:3,b in 0:1);
ospace = Rep[SU₂×U₁×ℤ₂]((i,j,b) => 2 for i in 0:1,j in -1:1,b in 0:1);
pspace = Rep[SU₂×U₁×ℤ₂]((i,j,b) => 1 for i in 1:1,j in 0:0,b in 0:0);

t_le = TensorMap(rand,ComplexF64,virtspace*ospace',virtspace);
t_re = TensorMap(rand,ComplexF64,virtspace*ospace,virtspace);
t_ac = TensorMap(rand,ComplexF64,virtspace*pspace,virtspace);
t_o = TensorMap(rand,ComplexF64,ospace*pspace,pspace*ospace);

factory = ac_eff(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(t_ac),space(t_ac)))

In [ ]:
@benchmark $factory(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

In [ ]:
@benchmark slowcontract(le = $t_le, re=$t_re, x = $t_ac, O = $t_o)

In [ ]:
norm(slowcontract(le = t_le, re=t_re, x = t_ac, O = t_o)- factory(le = t_le, re=t_re, x = t_ac, O = t_o))

# multithreaded benchmark

gc becomes a severe bottleneck with many julia threads

In [ ]:
virtspace = Rep[SU₂](0=>50, 1=> 30, 2=> 20, 3=>10, 4=> 5);
ospace = Rep[SU₂](0=>2, 1 => 2);
pspace = Rep[SU₂](1 => 1);


In [ ]:
numblocks = 1;
const leftenvs = [TensorMap(rand,ComplexF64,virtspace*ospace',virtspace) for i in 1:numblocks];
const rightenvs = [TensorMap(rand,ComplexF64,virtspace*ospace,virtspace) for i in 1:numblocks];
const mpos = [TensorMap(rand,ComplexF64,ospace*pspace,pspace*ospace) for i in 1:numblocks];

ac = TensorMap(rand,ComplexF64,virtspace*pspace,virtspace);

In [ ]:
using FLoops

t_le = leftenvs[1]
t_re = rightenvs[1]
t_o = mpos[1]

const fast_factory = ac_eff_malloc(le = (typeof(t_le),space(t_le)),re = (typeof(t_re),space(t_re)),O = (typeof(t_o),space(t_o)),x = (typeof(ac),space(ac)))

function fastfun(x)
    @floop for (l,m,r) in zip(leftenvs,mpos,rightenvs)
        y = fast_factory(le = l,re = r, O = m, x = x)
        @reduce s += y
    end
    s
end

function slowfun(x)
    @floop for (le,O,re) in zip(leftenvs,mpos,rightenvs)
        @tensor y[-1 -2;-3] := le[-1 2;1]*O[2 -2;3 4]*x[1 3;5]*re[5 4;-3]
        @reduce s += y
    end
    s
end

In [ ]:
@benchmark fastfun(ac)

In [ ]:
@benchmark slowfun(ac)

In [ ]:
using Profile
Profile.clear()
@profile fast_factory(le = t_le,re = t_re, O = t_o, x = ac)

In [ ]:
using ProfileVega
ProfileVega.view()